<a href="https://colab.research.google.com/github/JeanMusenga/PhD-Thesis_2024_Musenga/blob/main/BERTSum_SummarizationWithoutPreprocessingText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://chatgpt.com/share/c64e87ae-aefb-4eef-af28-94d2025276d7

To create a more effective extractive summarization approach using the BertSum model, we need to properly integrate the model's capabilities. Here’s a more structured and debugged approach:

1.Use the BERT model to encode the text.
2.Score each sentence using the BERT model.
3.Select the top sentences based on their scores.
We will ensure that the sentences are correctly indexed and processed.

https://github.com/nlpyang/BertSum


https://github.com/sudharsan13296/Getting-Started-with-Google-BERT/blob/main/6.%20Exploring%20BERTSUM%20for%20text%20summarization/6.07.%20Training%20the%20BERTSUM%20model%20.ipynb

In [ ]:
# Step 1: Install necessary dependencies
!pip install transformers
!pip install torch
!pip install nltk
!pip install openpyxl

In [2]:
# Step 2: Import necessary libraries
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from transformers import BertTokenizer, BertModel
import torch
from torch.nn.functional import softmax

In [3]:
# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
file_path = './saved_file'
file_path = ('DataSampePilot.xlsx')

data = pd.read_excel(file_path)

In [ ]:
# Display the first few rows of the dataset
data.head()

# Clone the Repository and Navigate to It
https://chatgpt.com/share/d9a17184-b011-4f11-85e4-7ab2636bbd30

In [5]:
!git clone https://github.com/nlpyang/BertSum.git


Cloning into 'BertSum'...
remote: Enumerating objects: 301, done.
remote: Counting objects: 100% (293/293), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 301 (delta 165), reused 290 (delta 164), pack-reused 8
Receiving objects: 100% (301/301), 15.05 MiB | 15.33 MiB/s, done.
Resolving deltas: 100% (165/165), done.


In [6]:
# List the files in the current directory
!ls

BertSum  DataSampePilot.xlsx  sample_data


In [7]:
!ls BertSum

bert_config_uncased_base.json  json_data  logs	  raw_data   results  urls
bert_data		       LICENSE	  models  README.md  src


In [6]:
%cd BertSum

/content/BertSum


# Step 3: Define the BertSum model for extractive summarization

In [7]:
# Step 4: Define the BertSum model for extractive summarization
class BertSum:
    def __init__(self, model_name='bert-base-uncased'):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertModel.from_pretrained(model_name)

    def summarize(self, text):
        sentences = sent_tokenize(text)
        if not sentences:
            return ""
        inputs = self.tokenizer(sentences, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs)
        sentence_embeddings = outputs.last_hidden_state.mean(dim=1)
        scores = torch.norm(sentence_embeddings, dim=1)
        top_sentence_idxs = scores.topk(3).indices.tolist()
        summary = '. '.join([sentences[idx] for idx in top_sentence_idxs])
        return summary

In [8]:
# Initialize the model
bertsum = BertSum()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [39]:
# Step 4: Apply the model to the Question_body and Answer_body columns
#data['Question_summary'] = data['Question_body'].apply(lambda x: bertsum.summarize(x) if pd.notnull(x) else "")
#data['Answer_summary'] = data['Answer_body'].apply(lambda x: bertsum.summarize(x) if pd.notnull(x) else "")


In [9]:
# Step 5: Apply the model to the Question_body and Answer_body columns
def summarize_column(text, column_name):
    if pd.notnull(text):
        try:
            summary = bertsum.summarize(text)
            print(f"Original {column_name}: {text[:100]}...")  # Print first 100 characters
            print(f"Summary: {summary}\n")
            return summary
        except Exception as e:
            print(f"Error summarizing text: {e}")
            return ""
    return ""

data['Question_summary'] = data['Question_body'].apply(lambda x: summarize_column(x, 'Question_body'))
data['Answer_summary'] = data['Answer_body'].apply(lambda x: summarize_column(x, 'Answer_body'))

Original Question_body: Kinda new to AWS. I have this high-level question. Iâ€™m looking for some insights about the general...
Summary: Thanks!. Would it be all about querying some API over HTTP (with Axios or Fetch API or whatever) or would it be some NPM package that queries a queue, in which case I might not even need Lambda checking for a return message, like Amplify from aws-amplify..?. Kinda new to AWS.

Original Question_body: I have some spring boot microservices and I want to use these microservices from a client applicatio...
Summary: If I use Spring MVC for designing the client side application, and from there, the client side application sends requests to the microservices REST APIs to use their services, is this a standard and correct solution?. <a href="https://i.sstatic.net/BnByRPzu.png" rel="nofollow noreferrer">This image better shows the issue:</a>. But the main logic of the application resides in the spring boot microservices.

Original Question_body: I'm trying to 

In [10]:
# Display the summarized data
data[['Question_body', 'Question_summary', 'Answer_body', 'Answer_summary']].head()


,Question_body,Question_summary,Answer_body,Answer_summary
0,Kinda new to AWS. I have this high-level quest...,Thanks!. Would it be all about querying some A...,"You send a request, you get a response. In ord...",polling some results).. Status of the flow (=t...
1,I have some spring boot microservices and I wa...,If I use Spring MVC for designing the client s...,<blockquote>\ntl;dr: Spring MVC will not contr...,E.g.. </blockquote>. Any basic REST API can en...
2,I'm trying to properly design an application a...,RemoteData - loads and caches data from the AP...,Determining the source of the information is b...,Put in an interface for RemoteUserContactsData...
3,I heard that for .NET8 Microsoft gifted us wit...,But that's trouble for another time.. <a href=...,I have always asked myself this very same ques...,":-). Easy Auth, using the &quot;Custom Provide..."
4,"I am trying to learn AWS services, and now it ...",Expose service as REST endpoints\nFeature like...,"Short answer is: no, you don't have to but you...",Alternative would be to implement your own gat...


In [ ]:
# Verify summarization by printing some samples
print(data[['Question_body', 'Question_summary']].head())
print(data[['Answer_body', 'Answer_summary']].head())

# Save the summarized data to a new Excel file

In [12]:
# Save the summarized data to a new Excel file
# Save to Excel
output_path = '/content/SummarizedData.xlsx'
data.to_excel(output_path, index=False, engine='openpyxl')

# Verify that the file has been saved correctly

In [ ]:
saved_data = pd.read_excel(output_path)
print("Saved summarized data:")
print(saved_data[['Question_summary', 'Answer_summary']].head())